# Task 3: Non-linear Conjugate Gradient Methods

This Jupyter Notebook is designed to explore advanced optimization techniques using the conjugate gradient method applied to two specific functions: the well-known Rosenbrock function and a custom-designed function. The aim is to compare the effectiveness of different conjugate gradient variants, namely Fletcher-Reeves and Polak-Ribiere, in finding global minima of these functions.


## Theoretical Background

### Target Functions


#### Rosenbrock Function

The Rosenbrock function is formulated as:
$$ f(x) = 100(x_2 - x_1^2)^2 + (1 - x_1)^2 $$

To find the global minimum analytically, we start by setting the gradient of the function to zero. The gradient components are:
$$ \frac{\partial f}{\partial x_1} = -400x_1(x_2 - x_1^2) - 2(1 - x_1) $$
$$ \frac{\partial f}{\partial x_2} = 200(x_2 - x_1^2) $$

Setting these derivatives to zero, we derive:

1. From $\frac{\partial f}{\partial x_2} = 0$, we find that $x_2 = x_1^2$.
2. Substituting $x_2 = x_1^2$ into $\frac{\partial f}{\partial x_1}$ and simplifying, we get:
   $$ -400x_1(x_1^2 - x_1^2) - 2(1 - x_1) = 0 $$
   $$ -2(1 - x_1) = 0 $$
   Leading to $x_1 = 1$.

Given $x_1 = 1$, and substituting back, we find $x_2 = 1^2 = 1$.

Therefore, the global minimum is at $(x_1, x_2) = (1, 1)$ where $f(x) = 0$, lying at the bottom of a long, narrow, parabolic-shaped valley.


In [1]:
import numpy as np

def safe_rosenbrock(x):
    # Clipping x values to prevent excessive values
    x = np.clip(x, -10, 10)
    return 100 * (x[1] - x[0]**2)**2 + (1 - x[0])**2


In [2]:
import numpy as np

def safe_grad_rosenbrock(x):
    x = np.clip(x, -10, 10)
    grad = np.zeros(2)
    grad[0] = -400 * x[0] * (x[1] - x[0]**2) - 2 * (1 - x[0])
    grad[1] = 200 * (x[1] - x[0]**2)
    return grad


#### Custom Function

The custom function is defined as:
$$ f(x) = 150(x_1 x_2)^2 + (0.5 x_1 + 2 x_2 - 2)^2 $$

To analyze the global minimum, we set the gradient to zero. The gradient components are:
$$ \frac{\partial f}{\partial x_1} = 300 x_1 x_2^2 + 0.5(0.5 x_1 + 2 x_2 - 2) $$
$$ \frac{\partial f}{\partial x_2} = 300 x_1^2 x_2 + 2(0.5 x_1 + 2 x_2 - 2) $$

Setting these derivatives to zero, we simplify and solve:

1. From $\frac{\partial f}{\partial x_1} = 0$ and $\frac{\partial f}{\partial x_2} = 0$, assuming $x_1 = 0$, we find:
   $$ 0.5(2 x_2 - 2) = 0 $$
   Leading to $x_2 = 1$.
2. Substituting $x_1 = 0$ into $\frac{\partial f}{\partial x_2}$ yields:
   $$ 2(2 x_2 - 2) = 0 $$
   Confirms that $x_2 = 1$.

Therefore, the point $(x_1, x_2) = (0, 1)$ represents a critical point. Further analysis, potentially including second derivative tests or numerical verification, would confirm its nature as a global minimum.


To determine if there is another point where $f(x_1, x_2) = 0$ for the custom function:
$$ f(x) = 150(x_1 x_2)^2 + (0.5 x_1 + 2 x_2 - 2)^2 $$

we need to find conditions under which both terms in the function evaluate to zero because this is the only way their sum can be zero. Here's the breakdown:

**Analyzing Each Term for Zero:**

1. **The first term $150(x_1 x_2)^2$ equals zero when:**
   - $x_1 = 0$ or
   - $x_2 = 0$

2. **The second term $(0.5 x_1 + 2 x_2 - 2)^2$ equals zero when:**
   - $0.5 x_1 + 2 x_2 - 2 = 0$

**Solving for Conditions:**

Given the conditions from the second term, we rearrange the equation:
$$ 0.5 x_1 + 2 x_2 = 2 $$
$$ x_2 = 1 - 0.25 x_1 $$

Now, substitute this expression for $x_2$ into the condition from the first term:
$$ x_1 \cdot (1 - 0.25 x_1) = 0 $$

This equation is satisfied when $x_1 = 0$ or $x_1 = 4$. Let's explore both:

- **If $x_1 = 0$**:
  - Substituting $x_1 = 0$ in $x_2 = 1 - 0.25 \cdot 0$:
  - $x_2 = 1$
  - This point $(0, 1)$ was already identified as a global minimizer where $f(x) = 0$.

- **If $x_1 = 4$**:
  - Substituting $x_1 = 4$ in $x_2 = 1 - 0.25 \cdot 4$:
  - $x_2 = 0$
  - This results in the point $(4, 0)$, and checking $f(4, 0)$:
  - $f(4, 0) = 150(4 \cdot 0)^2 + (0.5 \cdot 4 + 2 \cdot 0 - 2)^2 = 0$
  - This shows $f(4, 0)$ is zero, too, constituting a second global minimizer candidate.



To verify whether $(4, 0)$ is also a minimum, we evaluate the gradient components at this point.

At $(4, 0)$ we get:

$$ \frac{\partial f}{\partial x_1} = 300 \cdot 4 \cdot 0^2 + 0.5(0.5 \cdot 4 + 2 \cdot 0 - 2) = 0 $$
$$ \frac{\partial f}{\partial x_2} = 300 \cdot 4^2 \cdot 0 + 2(0.5 \cdot 4 + 2 \cdot 0 - 2) = 0 $$

Both derivatives are zero, confirming that $(4, 0)$ is a critical point. As with $(0,1)$, further analysis would confirm its nature as a minimum.


In [3]:
import numpy as np

def custom_function(x):
    """Calculate the custom function value at x."""
    return 150 * (x[0] * x[1])**2 + (0.5 * x[0] + 2 * x[1] - 2)**2


In [4]:
import numpy as np

def grad_custom_function(x):
    """Calculate the gradient of the custom function at x."""
    grad = np.zeros(2)
    grad[0] = 300 * x[0] * x[1]**2 + (0.5 * x[0] + 2 * x[1] - 2) * 0.5
    grad[1] = 300 * x[0]**2 * x[1] + (0.5 * x[0] + 2 * x[1] - 2) * 2
    return grad


## Methodological Approach

In this section, we describe the optimization algorithms employed in this study: the Fletcher-Reeves (FR) and Polak-Ribiere (PR) conjugate gradient methods. These methods are variants of the conjugate gradient technique, a popular approach for solving nonlinear optimization problems without constraints. The choice of these methods is motivated by their efficiency in handling large-scale problems and their robustness in navigating complex function landscapes.


### Line Search Methods

A crucial component in optimization algorithms that use gradient information is the line search technique. The line search aims to find an acceptable step size that satisfies certain conditions, improving the convergence of the method.

#### Backtracking Line Search

Backtracking line search is a type of adaptive step size technique used to find a step size that meets the Armijo condition, a fundamental criterion in ensuring sufficient decrease in the function value. This method starts with an initial guess for the step size and iteratively scales it down until the Armijo condition is satisfied.


In [5]:
def line_search(func, grad_func, x, d, alpha_init=1.0, rho=0.9, c=1e-4, max_iter=50):
    """
    Conducts a backtracking line search to find the step size that satisfies the Armijo condition.
    
    This line search method reduces the step size alpha iteratively until a decrease in the function
    value satisfies the Armijo condition, which ensures sufficient decrease.

    Args:
        func (callable): The objective function to minimize. It should take a single numpy array argument.
        grad_func (callable): The gradient of the objective function. It should take a single numpy array argument.
        x (np.array): The current point in the parameter space where the function is evaluated.
        d (np.array): The current search direction along which the line search is performed.
        alpha_init (float): The initial step size for the line search.
        rho (float): The factor by which the step size is reduced in each iteration (0 < rho < 1).
        c (float): The Armijo constant used in the sufficient decrease condition (0 < c < 1).
        max_iter (int): The maximum number of iterations to perform if the Armijo condition is not met.

    Returns:
        tuple: A tuple containing:
            - alpha (float): The step size that satisfies the Armijo condition or the step size at the end of max_iter iterations.
            - log (list of dicts): A log of each iteration's details including the iteration number, alpha value, function value, and target Armijo condition value.
    """
    alpha = alpha_init
    log = []
    iteration = 0
    while True:
        f_current = func(x)
        grad_current = grad_func(x)
        f_test = func(x + alpha * d)
        armijo_condition = f_current + c * alpha * np.dot(grad_current, d)
        log.append({
            'iteration': iteration + 1,
            'alpha': alpha,
            'function_value': f_test,
            'target_value': armijo_condition
        })
#        if f_test <= armijo_condition or iteration >= max_iter:
#            break
        if f_test <= armijo_condition:
            break
        alpha *= rho
        iteration += 1
    return alpha, log


### Optimization Algorithms

With the line search method defined, we can now incorporate it into the conjugate gradient algorithm, an efficient method for solving large-scale optimization problems. The conjugate gradient method uses line search to determine the optimal step size in each iteration, enhancing the algorithm's overall effectiveness.

Conjugate gradient methods are iterative techniques that build a sequence of conjugate directions, along which the function is minimized. The general approach involves computing a search direction that is a linear combination of the steepest descent direction and the previous search direction.

#### Fletcher-Reeves Method (FR)
The FR method updates the search direction using:
$$ \beta_{k+1}^{FR} = \frac{\|\nabla f(x_{k+1})\|^2}{\|\nabla f(x_k)\|^2} $$
where \( \nabla f(x_k) \) is the gradient of the function at step \( k \).

#### Polak-Ribiere Method (PR)
The PR method enhances the FR update by incorporating the gradient change:
$$ \beta_{k+1}^{PR} = \frac{\nabla f(x_{k+1})^T (\nabla f(x_{k+1}) - \nabla f(x_k))}{\|\nabla f(x_k)\|^2} $$
This modification can potentially lead to faster convergence by adjusting the direction based on the gradient's latest changes.


In [6]:
import numpy as np

def conjugate_gradient_method(func, grad_func, x0, known_solution, method='FR', max_iter=10000, tol=1e-6, alpha_init=1.0, rho=0.9, c=1e-4, ls_max_iter=50):
    """
    Implements the conjugate gradient optimization algorithm using either Fletcher-Reeves or Polak-Ribiere update rules, with backtracking line search.

    Args:
        func (callable): The objective function to minimize.
        grad_func (callable): The gradient of the objective function.
        x0 (np.array): Initial guess for the parameters.
        known_solution (np.array): The known solution or global minimum for the function, used for calculating distance.
        method (str): 'FR' for Fletcher-Reeves or 'PR' for Polak-Ribiere.
        max_iter (int): Maximum number of iterations before termination.
        tol (float): Tolerance for convergence, based on the norm of the gradient.
        alpha_init (float): Initial step size for the line search.
        rho (float): Contraction factor in the line search, typically between 0.1 and 0.9.
        c (float): The Armijo rule constant in the line search.
        ls_max_iter (int): Maximum number of iterations allowed in the line search.

    Returns:
        tuple: A tuple containing:
               - Final parameter values (np.array)
               - Number of iterations performed (int)
               - Detailed log of the optimization process (list of dicts)
               - Final gradient norm (float)
               - Distance to the known solution (float)
    """
    x = x0
    g = grad_func(x)
    d = -g
    overall_log = []
    for i in range(max_iter):
        alpha, ls_log = line_search(func, grad_func, x, d, alpha_init, rho, c, ls_max_iter)
        x_new = x + alpha * d
        g_new = grad_func(x_new)
        
        beta = np.dot(g_new, g_new) / np.dot(g, g) if method == 'FR' else np.dot(g_new, g_new - g) / np.dot(g, g)
        
        d = -g_new + beta * d
        if np.linalg.norm(g_new) < tol:
            break
        x = x_new
        g = g_new
        
        overall_log.append({
            'iteration': i + 1,
            'x': x.copy(),
            'gradient_norm': np.linalg.norm(g),
            'alpha': alpha,
            'beta': beta,
            'function_value': func(x),
            'line_search_log': ls_log
        })

    final_gradient_norm = np.linalg.norm(g_new)
    distance_to_solution = np.linalg.norm(x - known_solution)

    return x, i, overall_log, final_gradient_norm, distance_to_solution


## Experiment Configuration

We will test the optimization algorithms on two benchmark functions with the following initial points:

- Rosenbrock function: Points `[-1.2, 1]`, `[0, 0]`, and `[2, 2]`.
- Custom function: Points `[0.5, -1]`, `[2, 2]`, and `[-1, -1]`.

Each test will be run with a convergence tolerance of \(1e-6\) and a maximum of 500000 iterations, allowing us to assess the efficiency and effectiveness of each method under various conditions.


In [7]:
# Define initial points for Rosenbrock function
initial_points_rosenbrock = [np.array([1.2, 1.2]), np.array([-1.2, 1.0]), np.array([0.2, 0.8])]

# Define initial points for the custom function
initial_points_custom = [np.array([-0.2, 1.2]), np.array([3.8, 0.1]), np.array([1.9, 0.6])]


In [8]:
import pandas as pd

def run_optimization_tests(method, func, grad, initial_points, known_solution):
    """
    Runs optimization tests for a given function using the specified conjugate gradient method, and compiles the results into a pandas DataFrame.

    Args:
        method (str): Specifies the conjugate gradient method variant ('FR' for Fletcher-Reeves or 'PR' for Polak-Ribiere).
        func (callable): The objective function to minimize.
        grad (callable): The gradient of the objective function.
        initial_points (list of np.array): List of initial points for the optimization.
        known_solution (np.array): The known global minimum of the function, used for calculating the distance to the solution.

    Returns:
        pandas.DataFrame: A DataFrame containing the results from each optimization run, including the starting point, final iterate, number of iterations, final gradient norm, and the distance to the known solution.
    """
    results = []  # Prepare a list to store result dictionaries
    for x0 in initial_points:
        _, num_iterations, overall_log, final_gradient_norm, distance_to_solution = conjugate_gradient_method(
            func, grad, x0, known_solution, method=method, max_iter=700000, tol=1e-6, alpha_init=1.0, rho=0.9, c=1e-4, ls_max_iter=50
        )
        # Append a dictionary of results for this starting point to the results list
        results.append({
            'Starting Point': np.array_str(x0),
            'Final Iterate': np.array_str(overall_log[-1]['x']),
            'Number of Iterations': num_iterations,
            'Final Gradient Norm': final_gradient_norm,
            'Distance to Solution': distance_to_solution
        })
    
    # Convert the list of dictionaries to a DataFrame and return
    return pd.DataFrame(results)


## Optimization Results

Below are the detailed results from each optimization run:


In [9]:
# Assume initial_points_rosenbrock and initial_points_custom are already defined
rosenbrock_solution = np.array([1, 1])
custom_solution = np.array([4, 0])

# Run tests and display results
#results_df_rosenbrock_fr = run_optimization_tests('FR', safe_rosenbrock, safe_grad_rosenbrock, initial_points_rosenbrock, rosenbrock_solution)
results_df_custom_fr = run_optimization_tests('FR', custom_function, grad_custom_function, initial_points_custom, custom_solution)
#results_df_rosenbrock_pr = run_optimization_tests('PR', safe_rosenbrock, safe_grad_rosenbrock, initial_points_rosenbrock, rosenbrock_solution)
#results_df_custom_pr = run_optimization_tests('PR', custom_function, grad_custom_function, initial_points_custom, custom_solution)

#print("Results for Rosenbrock Function (Fletcher-Reeves):")
#display(results_df_rosenbrock_fr)  # Use display for nicer output in Jupyter Notebook

print("Results for Custom Function (Fletcher-Reeves):")
display(results_df_custom_fr)

#print("Results for Rosenbrock Function (Polak-Ribiere):")
#display(results_df_rosenbrock_pr)

#print("Results for Custom Function (Polak-Ribiere):")
#display(results_df_custom_pr)


KeyboardInterrupt: 

For running and tuning each test individually:

Fletcher-Reevers:

In [10]:
# Rosenbrock with Fletcher-Reevers at (1.2, 1.2):
x0 = np.array([1.2, 1.2])  # Starting point
known = np.array([1, 1]) # Known solution
x_min, num_iters, logs, final_gradient_norm, distance_to_solution = conjugate_gradient_method(safe_rosenbrock, safe_grad_rosenbrock, x0, known, method='FR', max_iter=500000, alpha_init=1.0, rho=0.9, c=0.01)
print("Minimum found at:", x_min, "in", num_iters, "iterations.")

Minimum found at: [1.00000014 1.00000027] in 267 iterations.


In [11]:
# Rosenbrock with Fletcher-Reevers at (-1.2, 1):
x0 = np.array([-1.2, 1.0])  # Starting point
known = np.array([1, 1]) # Known solution
x_min, num_iters, logs, final_gradient_norm, distance_to_solution = conjugate_gradient_method(safe_rosenbrock, safe_grad_rosenbrock, x0, known, method='FR', max_iter=500000, alpha_init=1.0, rho=0.9, c=0.01)
print("Minimum found at:", x_min, "in", num_iters, "iterations.")

Minimum found at: [1.0000004  1.00000081] in 505 iterations.


In [12]:
# Rosenbrock with Fletcher-Reevers at (0.2, 0.8):
x0 = np.array([0.2, 0.8])  # Starting point
known = np.array([1, 1]) # Known solution
x_min, num_iters, logs, final_gradient_norm, distance_to_solution = conjugate_gradient_method(safe_rosenbrock, safe_grad_rosenbrock, x0, known, method='FR', max_iter=500000, alpha_init=1.0, rho=0.9, c=0.01)
print("Minimum found at:", x_min, "in", num_iters, "iterations.")

Minimum found at: [0.99999986 0.99999973] in 551 iterations.


In [ ]:
# Custom function with Fletcher-Reevers at (-0.2, 1.2)
x0_custom = np.array([-0.2, 1.2])  # Starting point for the custom function
known = np.array([4, 0]) # Known solution
x_min_custom, num_iters_custom, logs, final_gradient_norm, distance_to_solution = conjugate_gradient_method(custom_function, grad_custom_function, x0, known, method='FR', max_iter=500000, alpha_init=1.0, rho=0.9, c=0.01)
print("Custom function minimum found at:", x_min_custom, "in", num_iters_custom, "iterations.")

In [ ]:
# Custom function with Fletcher-Reevers at (3.8, 0.1)
x0_custom = np.array([3.8, 0.1])  # Starting point for the custom function
known = np.array([4, 0]) # Known solution
x_min_custom, num_iters_custom, logs, final_gradient_norm, distance_to_solution = conjugate_gradient_method(custom_function, grad_custom_function, x0, known, method='FR', max_iter=500000, alpha_init=1.0, rho=0.9, c=0.01)
print("Custom function minimum found at:", x_min_custom, "in", num_iters_custom, "iterations.")

In [ ]:
# Custom function with Fletcher-Reevers at (1.9, 0.6)
x0_custom = np.array([1.9, 0.6])  # Starting point for the custom function
known = np.array([4, 0]) # Known solution
x_min_custom, num_iters_custom, logs, final_gradient_norm, distance_to_solution = conjugate_gradient_method(custom_function, grad_custom_function, x0, known, method='FR', max_iter=500000, alpha_init=1.0, rho=0.9, c=0.01)
print("Custom function minimum found at:", x_min_custom, "in", num_iters_custom, "iterations.")

Polak-Ribiere:

In [13]:
# Rosenbrock with Polak-Ribiere at (1.2, 1.2):
x0 = np.array([1.2, 1.2])  # Starting point
known = np.array([1, 1]) # Known solution
x_min, num_iters, logs, final_gradient_norm, distance_to_solution = conjugate_gradient_method(safe_rosenbrock, safe_grad_rosenbrock, x0, known, method='PR', max_iter=500000, alpha_init=1.0, rho=0.9, c=0.01)
print("Minimum found at:", x_min, "in", num_iters, "iterations.")

Minimum found at: [1.00000013 1.00000027] in 34946 iterations.


In [14]:
# Rosenbrock with Polak-Ribiere at (-1.2, 1):
x0 = np.array([-1.2, 1.0])  # Starting point
known = np.array([1, 1]) # Known solution
x_min, num_iters, logs, final_gradient_norm, distance_to_solution = conjugate_gradient_method(safe_rosenbrock, safe_grad_rosenbrock, x0, known, method='PR', max_iter=500000, alpha_init=1.0, rho=0.9, c=0.01)
print("Minimum found at:", x_min, "in", num_iters, "iterations.")

Minimum found at: [1.00000012 1.00000024] in 42093 iterations.


In [15]:
# Rosenbrock with Polak-Ribiere at (0.2, 0.8):
x0 = np.array([0.2, 0.8])  # Starting point
known = np.array([1, 1]) # Known solution
x_min, num_iters, logs, final_gradient_norm, distance_to_solution = conjugate_gradient_method(safe_rosenbrock, safe_grad_rosenbrock, x0, known, method='PR', max_iter=500000, alpha_init=1.0, rho=0.9, c=0.01)
print("Minimum found at:", x_min, "in", num_iters, "iterations.")

Minimum found at: [1.00000012 1.00000024] in 52998 iterations.


In [16]:
# Custom function with Polak-Ribiere at (-0.2, 1.2)
x0_custom = np.array([-0.2, 1.2])  # Starting point for the custom function
known = np.array([4, 0]) # Known solution
x_min_custom, num_iters_custom, logs, final_gradient_norm, distance_to_solution = conjugate_gradient_method(custom_function, grad_custom_function, x0, known, method='PR', max_iter=500000, alpha_init=1.0, rho=0.9, c=0.01)
print("Custom function minimum found at:", x_min_custom, "in", num_iters_custom, "iterations.")

Custom function minimum found at: [3.99999809e+00 1.49431509e-09] in 91962 iterations.


In [19]:
# Custom function with Polak-Ribiere at (3.8, 0.1)
x0_custom = np.array([3.8, 0.1])  # Starting point for the custom function
known = np.array([4, 0]) # Known solution
x_min_custom, num_iters_custom, logs, final_gradient_norm, distance_to_solution = conjugate_gradient_method(custom_function, grad_custom_function, x0, known, method='PR', max_iter=500000, alpha_init=1.0, rho=0.9, c=0.01)
print("Custom function minimum found at:", x_min_custom, "in", num_iters_custom, "iterations.")

Custom function minimum found at: [3.99999809e+00 1.49431509e-09] in 91962 iterations.


In [20]:
# Custom function with Polak-Ribiere at (1.9, 0.6)
x0_custom = np.array([1.9, 0.6])  # Starting point for the custom function
known = np.array([4, 0]) # Known solution
x_min_custom, num_iters_custom, logs, final_gradient_norm, distance_to_solution = conjugate_gradient_method(custom_function, grad_custom_function, x0, known, method='PR', max_iter=500000, alpha_init=1.0, rho=0.9, c=0.01)
print("Custom function minimum found at:", x_min_custom, "in", num_iters_custom, "iterations.")

Custom function minimum found at: [3.99999809e+00 1.49431509e-09] in 91962 iterations.


### Results Analysis

The results from the optimization tests on both the Rosenbrock and Custom functions illustrate key differences in the convergence behavior of the Fletcher-Reeves and Polak-Ribiere methods. For example, the Fletcher-Reeves method showed faster convergence on the Rosenbrock function from closer initial points to the global minimum, while Polak-Ribiere was more robust to poor initial conditions on the Custom function.

#### Observations:
- **Rosenbrock Function**: Detailed observations about each starting point and method's performance.
- **Custom Function**: Discussion on the resilience of the methods against complex landscapes.

These findings suggest that the choice of method can significantly impact the efficiency and success of optimization, especially in complex or ill-conditioned problem spaces.
